In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

C:\Users\Matthew\Miniconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [ ]:
ticker="PTON"
OHLCV_daily_data = pdr.get_data_yahoo(ticker,start="1970-01-01")
OHLCV_daily_data = OHLCV_daily_data.reset_index(drop=False)


present_time = datetime.datetime.now()
timestamp = present_time.strftime("%m/%d/%Y")
OHLCV_daily_data.head()
columns = OHLCV_daily_data.columns
columns = [x.lower() for x in columns]
OHLCV_daily_data.columns = columns
OHLCV_daily_data.head()

#########################################################################################
#########################################################################################
##########                MOVING AVERAGES AND BOLLINGER BANDS                  ##########
#########################################################################################
#########################################################################################

OHLCV_daily_data["twenty_day"]= OHLCV_daily_data["close"].rolling(window=20).mean()
OHLCV_daily_data["fifty_day"]= OHLCV_daily_data["close"].rolling(window=50).mean()
OHLCV_daily_data["twoHundred_day"]= OHLCV_daily_data["close"].rolling(window=200).mean()
OHLCV_daily_data["bolupp"]=OHLCV_daily_data["twenty_day"]+(2*OHLCV_daily_data["close"].rolling(window=20).std())
OHLCV_daily_data["bollow"]=OHLCV_daily_data["twenty_day"]-(2*OHLCV_daily_data["close"].rolling(window=20).std())

#########################################################################################
#########################################################################################
##########                RELATIVE STRENGTH INDEX                              ##########
#########################################################################################
#########################################################################################

period=14
delta = OHLCV_daily_data["close"].diff() 
up, down = delta.copy(), delta.copy()

up[up < 0] = 0
down[down > 0] = 0
    
# Calculate the exponential moving averages (EWMA)
roll_up = up.ewm(com=period - 1, adjust=False).mean()
roll_down = down.ewm(com=period - 1, adjust=False).mean().abs()
    
# Calculate RS based on exponential moving average (EWMA)
rs = roll_up / roll_down   # relative strength =  average gain/average loss

rsi = 100-(100/(1+rs))
OHLCV_daily_data['RSI'] = rsi

#########################################################################################
#########################################################################################
##########      ASSIGN CANDLESTICK AND VOLUME GAIN LOSS DIRECTION              ##########
#########################################################################################
#########################################################################################

OHLCV_daily_data["direction"] = ["increasing" if OHLCV_daily_data.loc[i,"close"] >= OHLCV_daily_data.loc[i,"open"] else "decreasing" for i in OHLCV_daily_data.index]
OHLCV_daily_data["color"] = ["forestgreen" if OHLCV_daily_data.loc[i,"direction"] == "increasing" else "red" for i in OHLCV_daily_data.index]

#########################################################################################
#########################################################################################
##########                   ASSIGN DATE WINDOW                                ##########
#########################################################################################
#########################################################################################

y=OHLCV_daily_data["date"].iloc[-1].year
m=OHLCV_daily_data["date"].iloc[-1].month
d=OHLCV_daily_data["date"].iloc[-1].day
mask = (OHLCV_daily_data["date"] >= str(y-1)+"-"+str(11)+"-"+str(d)) & (OHLCV_daily_data["date"] <= timestamp)
OHLCV_daily_data['date'].iloc[0]
OHLCV_daily_data['date'] = OHLCV_daily_data['date'].dt.strftime('%m-%d-%Y')

#########################################################################################
#########################################################################################
##########              SET CANDLESTICK YAXIS BOUNDS                           ##########
#########################################################################################
#########################################################################################

subset = OHLCV_daily_data[['open','high','low','close','bolupp','bollow']].columns
candle_low = OHLCV_daily_data.loc[mask,subset].min().min()
candle_high = OHLCV_daily_data.loc[mask,subset].max().max()

#########################################################################################
#########################################################################################
##########                   SET RSI 70 / 30 LINES                             ##########
#########################################################################################
#########################################################################################

rsiupL =[OHLCV_daily_data.loc[mask,"date"].iloc[0],OHLCV_daily_data.loc[mask,"date"].iloc[-1]]
rsiupR =[70,70]

rsidownL =[OHLCV_daily_data.loc[mask,"date"].iloc[0],OHLCV_daily_data.loc[mask,"date"].iloc[-1]]
rsidownR =[30,30]
rsimidL =[OHLCV_daily_data.loc[mask,"date"].iloc[0],OHLCV_daily_data.loc[mask,"date"].iloc[-1]]
rsimidR =[50,50]

In [27]:
import plotly.graph_objects as go
import plotly as py
py.offline.init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

fig = make_subplots(rows=3,cols=1,shared_xaxes=True,vertical_spacing=0.02, row_width=[0.2,0.6,0.2])

# RSI
fig.add_trace(
    go.Scatter(
        name="RSI "+ "{:.2f}".format(OHLCV_daily_data["RSI"].iloc[-1]),
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["RSI"].loc[mask],
        line=dict(color='black', width=3),
        yaxis='y1',hovertemplate = 'RSI: %{y:$.2f}'+ "<extra></extra>"),row=1,col=1
)
# RSI 70
fig.add_trace(
    go.Scatter(
        name="RSI 70",
        mode='lines',showlegend=False,hoverinfo="none",
        x=rsiupL,y=rsiupR,line=dict(color='black', width=2),
        yaxis='y1'),row=1,col=1
)
# RSI 50
fig.add_trace(
    go.Scatter(
        name="RSI 50",
        mode='lines',showlegend=False,hoverinfo="none",
        x=rsimidL,y=rsimidR,line=dict(color='black', width=2,dash='dot'),yaxis='y1'),row=1,col=1
)
# RSI 30
fig.add_trace(
    go.Scatter(
        name="RSI 30",
        mode='lines',showlegend=False,hoverinfo="none",
        x=rsidownL,y=rsidownR,line=dict(color='black', width=2),yaxis='y1'),row=1,col=1
)

bollinecolor='rgba(220,220,220,0.31)'
# Bollinger Bands - upper
fig.add_trace(
    go.Scatter(
        fill=None,
        mode="lines",
        line_color=bollinecolor,
        yaxis='y21',
        name='bollup',
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["bolupp"].loc[mask],hoverinfo="none",
        showlegend=False),row=2,col=1
)

# Bollinger Bands - lower
fig.add_trace(
    go.Scatter(
        fill="tonexty",
        mode="lines",
        line_color=bollinecolor,
        fillcolor=bollinecolor,
        name='bollow',
        yaxis='y21',
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["bollow"].loc[mask], hoverinfo="none",
        showlegend=False),row=2,col=1
)
# Candlesticks
fig.add_trace(
    go.Candlestick(
        name = str(ticker),
        showlegend=False,
        x=OHLCV_daily_data["date"].loc[mask],
        open=round(OHLCV_daily_data["open"].loc[mask],2),
        high=round(OHLCV_daily_data["high"].loc[mask],2),
        low=round(OHLCV_daily_data["low"].loc[mask],2),
        close=round(OHLCV_daily_data["close"].loc[mask],2),
        increasing_line_color = "forestgreen",
        decreasing_line_color = "red",yaxis='y2'),row=2,col=1)

# 20 day SMA
fig.add_trace(
    go.Scatter(
        name="MA(20) "+ "{:.2f}".format(OHLCV_daily_data["twenty_day"].iloc[-1]),
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["twenty_day"].loc[mask],
        line=dict(color='yellow', width=2),yaxis='y2',hovertemplate = '20MA: %{y:$.2f}'+ "<extra></extra>"),row=2,col=1
    )

# 50 day SMA
fig.add_trace(
    go.Scatter(
        name="MA(50) "+ "{:.2f}".format(OHLCV_daily_data["fifty_day"].iloc[-1]),
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["fifty_day"].loc[mask],line=dict(color='blue', width=2),yaxis='y2',cliponaxis=True,hovertemplate = '50MA: %{y:$.2f}'+ "<extra></extra>"),row=2,col=1
    )

# 200 day SMA
fig.add_trace(
    go.Scatter(
        name="MA(200) "+ "{:.2f}".format(OHLCV_daily_data["twoHundred_day"].iloc[-1]),
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["twoHundred_day"].loc[mask],line=dict(color='purple', width=2),yaxis='y2',hovertemplate = '200MA: %{y:$.2f}'+ "<extra></extra>"),row=2,col=1
    )

# volume
fig.add_trace(
    go.Bar(
        name='volume',
        showlegend=False,
        x=OHLCV_daily_data["date"].loc[mask],
        y=OHLCV_daily_data["volume"].loc[mask],marker_color=OHLCV_daily_data["color"].loc[mask],yaxis='y3'), row=3,col=1
    )

fig.update_layout(yaxis2=dict(overlaying='y2',layer='below traces'))

fig.update_layout(
    height=600,
    width=1000,
    plot_bgcolor='rgba(254,254,254,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    title=dict(
        text=str(ticker)+". "+
        str(present_time.strftime("%Y-%m-%d %H:%M:%S"))+
        ". Open: "+"{:.2f}".format(OHLCV_daily_data['open'].iloc[-1])+
        ". High: "+"{:.2f}".format(OHLCV_daily_data['high'].iloc[-1])+
        ". Low: "+"{:.2f}".format(OHLCV_daily_data['low'].iloc[-1])+
        ". Close: "+"{:.2f}".format(OHLCV_daily_data['close'].iloc[-1])+
        ". Volume: "+"{:.2f}".format(OHLCV_daily_data['volume'].iloc[-1]),
        font=dict(family="Arial",size=18))
)

# Update yaxis properties
fig.update_yaxes(tickangle=0,
                 showline=True, linewidth=2, linecolor='black', mirror=True,
                 showgrid=False, gridwidth=1, gridcolor='black',
                 tickfont=dict(family='Arial', color='black', size=14),nticks=6,tickformat='', range=[10,90],row=1,col=1)
fig.update_yaxes(tickangle=0,
                 showline=True, linewidth=2, linecolor='black', mirror="allticks",
                 showgrid=True, gridwidth=1, gridcolor='black',
                 tickfont=dict(family='Arial', color='black', size=14),range =[candle_low-2,candle_high+2],
                 nticks=27,tickformat='$', row=2, col=1)

fig.update_yaxes(tickangle=0,
                 showline=True, linewidth=2, linecolor='black', mirror="ticks",
                 showgrid=True, gridwidth=1, gridcolor='black',
                 tickfont=dict(family='Arial', color='black', size=14),
                 row=3, col=1)


fig.update_layout(xaxis2_rangeslider_visible=False)
fig.update_xaxes(type='category',nticks=25,tickangle=45,
                 showline=True, linewidth=2, linecolor='black', mirror=True,
                 showgrid=True, gridwidth=.5, gridcolor='black', tickfont=dict(family='Arial', color='black', size=14))

fig.update_layout(
    legend=dict(
        x=0,
        y=1.1,
        traceorder="normal",
        font=dict(
            family="Arial",
            size=14,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2,orientation='h'
    )
)

fig.update_layout(hovermode="x")

fig.show()
fig.write_html(str(ticker)+".html")

In [5]:
50*7

350